In [1]:
import pandas as pd

In [2]:
# win probabilities outputted by W3_Predictions.py
submission = pd.read_csv('mydata/womens/original_probabilities.csv')
submission.head()

,ID,Pred,TeamID_x,TeamID_y,SeedDiff,Seed_x,Seed_y
0,2021_3104_3112,0.769516,3112,3104,-4,3,7
1,2021_3104_3116,0.541189,3116,3104,-3,4,7
2,2021_3104_3124,0.987117,3124,3104,-5,2,7
3,2021_3104_3163,0.992835,3163,3104,-6,1,7
4,2021_3104_3208,0.769516,3208,3104,-4,3,7


In [3]:
# function to set a min probability for the favorite based on the difference in seed
# used for 1 v 16, 2 v 15, 3 v 14, 4 v 13
def set_prob_to_k(row, k, diff):
    if (row['SeedDiff'] < diff) and (row['Pred'] < k):
        if diff == -10 and row['Seed_y'] < 14:
            return row['Pred']
        else:
            return k
    else:
        return row['Pred']
    
submission['Pred'] = submission.apply(set_prob_to_k, axis = 1, args = (.96, -8)) # 4 vs 13 set to minimum 96%
submission['Pred'] = submission.apply(set_prob_to_k, axis = 1, args = (1, -10)) # 3 v 14, 2 v 15 and 1 v 16 set to 100%

In [4]:
# Switches Pred to the (1 - Pred) if TeamID_x is not the lower ID as Pred needs to be the probability for the lower ID team
def switch_teams(row):
    idlist = row['ID'].split('_')
    prob = row['Pred']
    if row['TeamID_x'] != int(idlist[1]):
        row['Pred'] = 1 - prob
    row['TeamID_x'] = int(idlist[1])
    row['TeamID_y'] = int(idlist[2])
    return row

submission = submission.apply(switch_teams, axis = 1)

In [5]:
# File for names of teams
team_names = pd.read_csv('ncaaw-march-mania-2021/WTeams.csv')[['TeamID', 'TeamName']]
team_names.head()

,TeamID,TeamName
0,3101,Abilene Chr
1,3102,Air Force
2,3103,Akron
3,3104,Alabama
4,3105,Alabama A&M


In [6]:
# Seeds for 2021
seeds = pd.read_csv('ncaaw-march-mania-2021/WNCAATourneySeeds.csv').query('Season == 2021')[['TeamID', 'Seed']]
seeds.head()

,TeamID,Seed
1408,3390,W01
1409,3257,W02
1410,3208,W03
1411,3116,W04
1412,3283,W05


In [7]:
# join names with seeds
team_names_seeds = pd.merge(team_names, seeds, on = 'TeamID')
team_names_seeds.head()

,TeamID,TeamName,Seed
0,3104,Alabama,X07
1,3112,Arizona,Y03
2,3116,Arkansas,W04
3,3124,Baylor,Z02
4,3125,Belmont,Y12


In [8]:
# Tournament slots for 2021
slots = pd.read_csv('ncaaw-march-mania-2021/WNCAATourneySlots.csv')[['Slot', 'StrongSeed', 'WeakSeed']]
slots.head()

,Slot,StrongSeed,WeakSeed
0,R1W1,W01,W16
1,R1W2,W02,W15
2,R1W3,W03,W14
3,R1W4,W04,W13
4,R1W5,W05,W12


In [9]:
# join slots with names and seeds
team_slots = pd.merge(slots, team_names_seeds, left_on = ['StrongSeed'], right_on = ['Seed'])
team_slots = pd.merge(team_slots, team_names_seeds, left_on = ['WeakSeed'], right_on = ['Seed']).drop(columns = ['Slot', 'StrongSeed', 'WeakSeed', 'Seed_x', 'Seed_y'])
team_slots.head()

,TeamID_x,TeamName_x,TeamID_y,TeamName_y
0,3390,Stanford,3430,Utah Valley
1,3257,Louisville,3265,Marist
2,3208,Georgia,3180,Drexel
3,3116,Arkansas,3460,Wright St
4,3283,Missouri St,3413,UC Davis


In [10]:
# function to switch slots so the lower id is in first slot
def switch_slots(row):
    if row['TeamID_x'] > row['TeamID_y']:
        team_id = row['TeamID_x']
        name = row['TeamName_x']
        row['TeamID_x'] = row['TeamID_y']
        row['TeamID_y'] = team_id
        row['TeamName_x'] = row['TeamName_y']
        row['TeamName_y'] = name
    return row

team_slots = team_slots.apply(switch_slots, axis = 1)
team_slots.head()

,TeamID_x,TeamName_x,TeamID_y,TeamName_y
0,3390,Stanford,3430,Utah Valley
1,3257,Louisville,3265,Marist
2,3180,Drexel,3208,Georgia
3,3116,Arkansas,3460,Wright St
4,3283,Missouri St,3413,UC Davis


In [11]:
# merge submission with team slots to look at Round 1 probabilities
submission_r1 = pd.merge(submission, team_slots, on = ['TeamID_x', 'TeamID_y'])
submission_r1

,ID,Pred,TeamID_x,TeamID_y,SeedDiff,Seed_x,Seed_y,TeamName_x,TeamName_y
0,2021_3125_3211,0.151798,3125,3211,-7,5,12,Belmont,Gonzaga
1,2021_3133_3400,0.072379,3133,3400,-5,6,11,Bradley,Texas
2,2021_3140_3353,0.169489,3140,3353,-5,6,11,BYU,Rutgers
3,2021_3141_3234,0.125030,3141,3234,-7,5,12,C Michigan,Iowa
4,2021_3180_3208,0.000000,3180,3208,-11,3,14,Drexel,Georgia
5,2021_3195_3276,0.314285,3195,3276,-5,6,11,FL Gulf Coast,Michigan
6,2021_3199_3333,0.482404,3199,3333,-1,8,9,Florida St,Oregon St
7,2021_3210_3372,0.266021,3210,3372,7,12,5,Georgia Tech,SF Austin
8,2021_3163_3219,1.000000,3163,3219,-15,1,16,Connecticut,High Point
9,2021_3226_3246,0.040000,3226,3246,-9,4,13,Idaho St,Kentucky


In [12]:
# Call this function for all heavy favorites to make sure they have high probabilities

# prints the predicted probabilties for all games involving the given team id
def print_games(team_id):
    preds = []
    opp_seeds = []
    
    for i in range(len(submission)):
        id_list = submission.loc[i, 'ID'].split('_')
        prob = submission.loc[i, 'Pred']
        id_x = int(id_list[1])
        id_y = int(id_list[2])
        if id_x == team_id:
            opp_seeds.append(seeds[seeds['TeamID'] == id_y].reset_index().loc[0, 'Seed'])
            preds.append(prob)
        elif team_id == id_y:
            opp_seeds.append(seeds[seeds['TeamID'] == id_x].reset_index().loc[0, 'Seed'])
            preds.append((1 - prob))
    
    return pd.DataFrame({'Pred': preds, 'OppSeed': opp_seeds}).sort_values('Pred')

In [13]:
# UConn
pd.set_option('display.max_rows', 64)
print_games(3163)

,Pred,OppSeed
62,0.508450,W01
61,0.630470,Z02
56,0.809389,X02
58,0.858990,X01
57,0.951256,Y01
60,0.955602,X03
55,0.964407,W02
32,0.968081,W06
9,0.974107,W03
54,0.974107,Y03


In [14]:
# Stanford
print_games(3390)

,Pred,OppSeed
62,0.491550,Z01
61,0.610648,Z02
56,0.796060,X02
58,0.848481,X01
57,0.947200,Y01
60,0.951890,X03
55,0.961401,W02
32,0.965375,W06
9,0.971896,W03
54,0.971896,Y03


In [15]:
# Baylor
print_games(3124)

,Pred,OppSeed
61,0.369530,Z01
62,0.389352,W01
56,0.701328,X02
58,0.771097,X01
57,0.915195,Y01
60,0.922495,X03
55,0.937436,W02
32,0.943732,W06
9,0.954136,W03
54,0.954136,Y03


In [16]:
# South Carolina
print_games(3376)

,Pred,OppSeed
58,0.141010,Z01
60,0.151519,W01
57,0.228903,Z02
59,0.427666,X02
56,0.748729,Y01
62,0.766706,X03
55,0.805340,W02
32,0.822412,W06
9,0.851723,W03
54,0.851723,Y03


In [17]:
# Maryland
print_games(3268)

,Pred,OppSeed
57,0.190611,Z01
58,0.203940,W01
56,0.298672,Z02
60,0.572334,X01
59,0.810866,Y01
62,0.825434,X03
55,0.856167,W02
32,0.869504,W06
9,0.892062,W03
54,0.892062,Y03


In [18]:
# NC State
print_games(3301)

,Pred,OppSeed
57,0.048744,Z01
60,0.052800,W01
56,0.084805,Z02
58,0.189134,X02
59,0.251271,X01
62,0.502769,X03
55,0.560029,W02
32,0.587598,W06
9,0.638636,W03
54,0.638636,Y03


In [19]:
# UCLA
print_games(3417)

,Pred,OppSeed
58,0.044398,Z01
62,0.048110,W01
57,0.077505,Z02
59,0.174566,X02
61,0.233294,X01
60,0.497231,Y01
55,0.534685,W02
32,0.562602,W06
9,0.614704,W03
54,0.614704,Y03


In [20]:
# adjusts all probabiltiies for a given team
# positive adj means team is undervalued by model
# negative adj means team is overvalued by model
def team_adjustment(row, team_id, adj):
    idlist = row['ID'].split('_')
    
    # factor to add or subtract from the win probability, smaller at the tails than at the center and never goes outside of 0 and 1
    if row['Pred'] > 0.5:
        adj_factor = abs(adj) * (1 - row['Pred'])
    else:
        adj_factor = abs(adj) * row['Pred']
    
    # increase probability if given team is TeamID_x and adjustment is positive or given team is TeamID_y and adjustment is negative
    if (team_id == int(idlist[1]) and adj > 0) or (team_id == int(idlist[2]) and adj < 0):
        return row['Pred'] + adj_factor
        
    # decrease probability if given team is TeamID_y and adjustment is positive or given team is TeamID_x and adjustment is negative
    elif (team_id == int(idlist[2]) and adj > 0) or (team_id == int(idlist[1]) and adj < 0):
        return row['Pred'] - adj_factor
        
    else:
        return row['Pred']
    
# look at Vegas odds, and see which teams the model undervalues that make sense (injuries, COVID, other reasons)
# and adjust their win probabilities for all games manually using function above

submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3124, -0.5), axis = 1) # Baylor
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3199, -0.2), axis = 1) # FSU

In [21]:
submission.query('ID ==\'2021_3199_3333\'')

,ID,Pred,TeamID_x,TeamID_y,SeedDiff,Seed_x,Seed_y
345,2021_3199_3333,0.385923,3199,3333,-1,8,9


In [22]:
# Baylor
print_games(3124)

,Pred,OppSeed
61,0.184765,Z01
62,0.194676,W01
56,0.551992,X02
58,0.656646,X01
57,0.872793,Y01
60,0.883743,X03
55,0.906153,W02
32,0.915598,W06
9,0.931203,W03
54,0.931203,Y03


### Kaggle Alterations

Kaggle allows two submission files, so it's best to alter the submission probabilities to maximize your chance of winning. To do this, I will take the most even first round matchup, and assign a probability of 1 in the first submission and a probability of 0 in the second submission. This matchup is guaranteed to happen (as it's the opening round) and I will benefit from the improvement in log loss on that matchup going from a log loss of around 0.69 to a log loss of 0

In [23]:
submission.query('ID ==\'2021_3104_3314\'')

,ID,Pred,TeamID_x,TeamID_y,SeedDiff,Seed_x,Seed_y
1104,2021_3104_3314,0.48727,3104,3314,-3,7,10


In [24]:
# Assign probabiltiy 1 to the most even first round matchup
# juding from submission_r1 above, the most even first round matchup is...
submission1 = submission.copy()[['ID', 'Pred']]
submission2 = submission.copy()[['ID', 'Pred']]
submission1.iloc[1104, 1] = 1   # Alabama v UNC, give Alabama the full 100%
submission2.iloc[1104, 1] = 0   # Alabama v UNC, give UNC the full 100%

In [25]:
submission1.iloc[1104, :]

ID      2021_3104_3314
Pred                 1
Name: 1104, dtype: object

In [26]:
submission2.iloc[1104, :]

ID      2021_3104_3314
Pred                 0
Name: 1104, dtype: object

In [27]:
# write submissions to csv
submission.to_csv('mydata/womens/altered_probabilities.csv', index = False)
submission1.to_csv('mydata/womens/submission1_prob.csv', index = False)
submission2.to_csv('mydata/womens/submission2_prob.csv', index = False)